<a href="https://colab.research.google.com/github/Javier19-cmd/Lab5-SDS/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 5

Javier Valle 20159

In [10]:
#from Archivos.clasificador import clasificacion
from clasificador import clasificacion
import pandas as pd
import json
import random
import re
import numpy as np

In [ ]:
!pip install python-whois

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.1 MB/s eta 0:00:00


In [2]:
import whois

In [93]:
!pip install tldextract

In [3]:
import tldextract

### Parte 1

Leyendo y guardando el tráfico que es DNS nada más

In [9]:
with open('./large_eve.json', 'r') as archivo_json, \
        open('./dns_trafico.json', 'w') as archivo_dns:

    for linea in archivo_json:
        datos = json.loads(linea)
        if datos['event_type'] == 'dns':
            json.dump(datos, archivo_dns)
            archivo_dns.write('\n')

archivo_json.close()
archivo_dns.close()

Enseñando la cantidad de registros

In [5]:
with open('./dns_trafico.json', 'r') as archivo_dns:
    contador_registros = 0

    for linea in archivo_dns:
        contador_registros += 1

print("Cantidad de registros guardados:", contador_registros)

Cantidad de registros guardados: 15749


Enseñando la información de 2 registros cualesquiera

In [6]:
lineas = []

with open('./dns_trafico.json', 'r') as archivo_dns:
    for linea in archivo_dns:
        lineas.append(linea)

# Escogiendo dos índices aleatorios dentro del rango de líneas en el archivo
indice1 = random.randint(0, len(lineas)-1)
indice2 = random.randint(0, len(lineas)-1)

# Obteniendo los datos de los registros correspondientes a los índices escogidos
registro1 = json.loads(lineas[indice1])
registro2 = json.loads(lineas[indice2])


print("Registro 1:")
print(json.dumps(registro1, indent=4))
print("\nRegistro 2:")
print(json.dumps(registro2, indent=4))


Registro 1:
{
    "timestamp": "2017-07-22T18:01:27.700260-0500",
    "flow_id": 540714828541796,
    "pcap_cnt": 279096,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "192.168.201.68",
    "src_port": 55770,
    "dest_ip": "192.168.207.4",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 58371,
        "rrname": "secure.informaction.com.localdomain",
        "rrtype": "A",
        "tx_id": 0
    }
}

Registro 2:
{
    "timestamp": "2017-07-22T18:29:18.489627-0500",
    "flow_id": 39470779758547,
    "pcap_cnt": 506573,
    "event_type": "dns",
    "vlan": 150,
    "src_ip": "192.168.207.4",
    "src_port": 53,
    "dest_ip": "192.168.205.188",
    "dest_port": 40118,
    "proto": "UDP",
    "dns": {
        "type": "answer",
        "id": 19780,
        "rcode": "NXDOMAIN",
        "rrname": "mirrors.ecvps.com"
    }
}


Normalizando la data

In [7]:
with open('./dns_trafico.json', 'r') as archivo_dns:
    datos = [json.loads(linea) for linea in archivo_dns]

# Normalizando los datos
df = pd.json_normalize(datos)

print("Shape del DataFrame:", df.shape)

Shape del DataFrame: (15749, 18)


Buscando dominios DGA

In [8]:
# Filtrando los registros DNS tipo A
registros_tipo_A = df[df['dns.rrtype'] == 'A']

print("Shape del DataFrame filtrado:", len(registros_tipo_A))

Shape del DataFrame filtrado: 2849


Filtrando los dominios únicos

In [12]:
# Conjunto para almacenar dominios únicos
unique_domains = set()

with open('./dns_trafico.json', 'r') as archivo_dns:
    for linea in archivo_dns:
        datos = json.loads(linea)
        # Extrayendo el dominio del campo 'rrname' y agregarlo al conjunto
        domain = datos['dns']['rrname']
        unique_domains.add(domain)

num_unique_domains = len(unique_domains)

print("Cantidad de registros únicos:", num_unique_domains)

Cantidad de registros únicos: 334


Escribiendo una función que obtengo el TLD para un dominio

In [14]:
def obtener_TLD(dominio):
    # Expresión regular para encontrar el TLD
    patron = r'\.(.*?)\.(?:com|org|net|gov|edu|int|mil|arpa|biz|info|name|pro|aero|coop|museum|jobs|travel|cat|tel|onion|git|home|local|example|test|invalid|localhost|localdomain)$'
    match = re.search(patron, dominio)
    if match:
        return match.group(1)
    else:
        return dominio

# Ejemplo de uso
dominio1 = "api.wunderground.com"
dominio2 = "safebrowsing.clients.google.com.home"

print("TLD de", dominio1, ":", obtener_TLD(dominio1))
print("TLD de", dominio2, ":", obtener_TLD(dominio2))

TLD de api.wunderground.com : wunderground
TLD de safebrowsing.clients.google.com.home : clients.google.com


Prompt utilizado: "hola, me podrías escribir una función que me obtenga el TLD para un dominio"

Creando una columna que se llame domain_tld y eliminando las demás columnas

In [15]:
if 'domain_tld' not in df.columns:
    df['domain_tld'] = df['dns.rrname'].apply(obtener_TLD)

    df = df[['domain_tld']]

print(df)

                        domain_tld
0                     wunderground
1                          dropbox
2      office.aol.com.ad.aol.aoltw
3      office.aol.com.ad.aol.aoltw
4                           <root>
...                            ...
15744               start3.mozilla
15745                       <root>
15746        whitecell.localdomain
15747        whitecell.localdomain
15748                       <root>

[15749 rows x 1 columns]


### Parte 2

Usando el clasificador proporcionado y creando un nuevo dataframe

In [16]:
df

result = clasificacion(df)

result

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,domain_tld,isDGA
0,wunderground,1
1,dropbox,0
2,office.aol.com.ad.aol.aoltw,1
3,office.aol.com.ad.aol.aoltw,1
4,<root>,0
...,...,...
15744,start3.mozilla,1
15745,<root>,0
15746,whitecell.localdomain,0
15747,whitecell.localdomain,0


Filtrando aquellos que son DGA y enseñándolos.

In [17]:
# Filtrando aquellos considerados como DGA (valor 1)
result = result[result['isDGA'] == 1]

print("Registros considerados como DGA:")
print(result)

# Eliminando duplicados
result_unique = result.drop_duplicates()


print("\nCantidad de registros únicos después de eliminar duplicados:", len(result_unique))

Registros considerados como DGA:
                              domain_tld  isDGA
0                           wunderground      1
2            office.aol.com.ad.aol.aoltw      1
3            office.aol.com.ad.aol.aoltw      1
5                           wunderground      1
21           office.aol.com.ad.aol.aoltw      1
...                                  ...    ...
15735  _dns-sd._udp.0.195.16.172.in-addr      1
15740                 26.168.192.in-addr      1
15741                 26.168.192.in-addr      1
15743                     start3.mozilla      1
15744                     start3.mozilla      1

[3525 rows x 2 columns]

Cantidad de registros únicos después de eliminar duplicados: 71


### Parte 3 - Dominio experto

hola, me podrías hacer un código que lo que haga es obtener todos los tld de un dataset como este


id,domain
1,google.com
2,www.google.com
3,microsoft.com
4,data.microsoft.com
5,events.data.microsoft.com
6,live.com
7,office.com
8,apple.com

en donde su resultado debe ser 0 si el TLD se encuentra en la list y 1 si no existe.

In [18]:
data = pd.read_csv("top-1m.csv")

dataframe = pd.DataFrame(data)

print(result_unique)

allowed_tlds = set(result_unique["domain_tld"])

# Función para verificar si el TLD está permitido
def check_tld(domain):
    if pd.isnull(domain):  # Manejando valores nulos
        return 1
    tld = domain.split('.')[-1]
    if tld in allowed_tlds:
        return 0
    else:
        return 1

                               domain_tld  isDGA
0                            wunderground      1
2             office.aol.com.ad.aol.aoltw      1
81                     21.168.192.in-addr      1
94                     22.168.192.in-addr      1
117    _dns-sd._udp.0.108.168.192.in-addr      1
...                                   ...    ...
10607                     192.168.21-28.0      1
10646                ejfodfmfxlkgifuf.xyz      1
14730                         168.21-28.0      1
15507                     192.168.22.201:      1
15716              168.22.201:.stayonline      1

[71 rows x 2 columns]


<ipython-input-18-4530e1c87829>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("top-1m.csv")


Usar la función para determinan si los TLD se encuentran en dicha lista. Filtrar aquellos que si se encuentran.

In [19]:
# Aplicando la función a la columna 'domain' del DataFrame
dataframe['result'] = dataframe['domain'].apply(check_tld)

filtered_df = dataframe[dataframe['result'] == 0]

filtered_df = filtered_df.drop_duplicates(subset=['domain'])

print(filtered_df)

print("Cantidad de resultados: ")
print(len(filtered_df))

             id                             domain  result
1394       1395                    cloud.microsoft       0
2286       2287                   static.microsoft       0
3113       3114            outlook.cloud.microsoft       0
3121       3122              teams.cloud.microsoft       0
3666       3667     otelrules.svc.static.microsoft       0
...         ...                                ...     ...
810555   778969               loop.cloud.microsoft       0
811771   780185     rssfeed.azure.status.microsoft       0
969734   938148  nam.sessions.mesh.cloud.microsoft       0
1009442  977856            hrd.svc.cloud.microsoft       0
1010652  979066       msit.hrd.svc.cloud.microsoft       0

[83 rows x 3 columns]
Cantidad de resultados: 
83


Buscando la fecha de creación del TLD en base a este dataset

            id                             domain  result
1394       1395                    cloud.microsoft       0
2286       2287                   static.microsoft       0
3113       3114            outlook.cloud.microsoft       0
3121       3122              teams.cloud.microsoft       0
3666       3667     otelrules.svc.static.microsoft       0
...         ...                                ...     ...
810555   778969               loop.cloud.microsoft       0
811771   780185     rssfeed.azure.status.microsoft       0
969734   938148  nam.sessions.mesh.cloud.microsoft       0
1009442  977856            hrd.svc.cloud.microsoft       0
1010652  979066       msit.hrd.svc.cloud.microsoft       0

[83 rows x 3 columns]
Cantidad de resultados:
83

In [28]:
def get_creation_date(tld):
    try:
        domain = whois.whois(tld)
        return domain.creation_date
    except Exception as e:
        print(f"Error obteniendo información WHOIS para {tld}: {e}")
        return None

In [26]:
filtered_df

,id,domain,result,tld,creation_date
1394,1395,cloud.microsoft,0,microsoft,None
2286,2287,static.microsoft,0,microsoft,None
3113,3114,outlook.cloud.microsoft,0,microsoft,None
3121,3122,teams.cloud.microsoft,0,microsoft,None
3666,3667,otelrules.svc.static.microsoft,0,microsoft,None
...,...,...,...,...,...
810555,778969,loop.cloud.microsoft,0,microsoft,None
811771,780185,rssfeed.azure.status.microsoft,0,microsoft,None
969734,938148,nam.sessions.mesh.cloud.microsoft,0,microsoft,None
1009442,977856,hrd.svc.cloud.microsoft,0,microsoft,None


Enseñando la fecha de creación

In [29]:
# Aplicar la función para obtener la fecha de creación de cada TLD en el DataFrame
filtered_df['creation_date'] = filtered_df['tld'].apply(get_creation_date)

# Imprimir el DataFrame con la fecha de creación de cada TLD
filtered_df[['tld', 'creation_date']]

,tld,creation_date
1394,microsoft,None
2286,microsoft,None
3113,microsoft,None
3121,microsoft,None
3666,microsoft,None
...,...,...
810555,microsoft,None
811771,microsoft,None
969734,microsoft,None
1009442,microsoft,None


En este caso, podemos confirmar que todos son sospechosos, dado que no se pudo obtener información de la fecha de creación del link.

Indicar cuales dominios tienen secuencias aleatorias de caracteres, no palabras.

In [55]:
def detect_dga(domain):
    # Verificando si el dominio contiene solo caracteres alfanuméricos
    if not domain.isalnum():
        return True

    # Verificando si el dominio contiene solo consonantes
    vowels = 'aeiou'
    consonants = 'bcdfghjklmnpqrstvwxyz'
    if all(c.lower() in consonants for c in domain):
        return True

    # Si no se cumple ninguna de las condiciones anteriores, el dominio no se considera sospechoso de DGA
    return False

In [56]:
filtered_df["is_dga"] = filtered_df['domain'].apply(detect_dga)

suspicious_domains = filtered_df[filtered_df['is_dga']]
print("Dominios sospechosos de DGA:")
print(suspicious_domains[['domain', 'is_dga']])

Dominios sospechosos de DGA:
                                    domain  is_dga
1394                       cloud.microsoft    True
2286                      static.microsoft    True
3113               outlook.cloud.microsoft    True
3121                 teams.cloud.microsoft    True
3666        otelrules.svc.static.microsoft    True
...                                    ...     ...
810555                loop.cloud.microsoft    True
811771      rssfeed.azure.status.microsoft    True
969734   nam.sessions.mesh.cloud.microsoft    True
1009442            hrd.svc.cloud.microsoft    True
1010652       msit.hrd.svc.cloud.microsoft    True

[83 rows x 2 columns]


En este caso, se puede notar que todos los dominios son sospechosos, debido a que presentan alguna incoherencia, como lo podría ser, por ejemplo, azure.status, o mist.hrd.svc.clout.